In [1]:
from sklearn import svm, datasets
iris = datasets.load_iris()

In [2]:
import pandas as pd
df = pd.DataFrame(iris.data,columns=iris.feature_names)
df['flower'] = iris.target
df['flower'] = df['flower'].apply(lambda x: iris.target_names[x])
df[47:150]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,setosa
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [3]:
#Approach 1: Use train_test_split and manually tune parameters by trial and error

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3)

In [4]:
model = svm.SVC(kernel='rbf',C=30,gamma='auto')
model.fit(X_train,y_train)
model.score(X_test, y_test)

0.8888888888888888

In [9]:
#Approach 2: Use K Fold Cross validation
#Manually try suppling models with different parameters to cross_val_score function with 5 fold cross validation
from sklearn.model_selection import cross_val_score

cross_val_score(svm.SVC(kernel='rbf',C=30,gamma='auto'),iris.data, iris.target,cv=3)

array([0.98, 0.94, 0.96])

In [11]:
#Above approach is tiresome and very manual. We can use for loop as an alternative
import numpy as np
kernels = ['rbf', 'linear']
C = [1,10,20]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel=kval,C=cval,gamma='auto'),iris.data, iris.target, cv=5)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)

avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

In [16]:
# Approach 3: Use GridSearchCV
# GridSearchCV does exactly same thing as for loop above but in a single line of code
from sklearn.model_selection import GridSearchCV
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10,20]}
clf= GridSearchCV(svm.SVC(),parameters)

In [18]:
clf.fit(iris.data, iris.target)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10, 20], 'kernel': ('linear', 'rbf')})

In [20]:
clf.best_estimator_

SVC(C=1, kernel='linear')

In [21]:
clf.best_params_

{'C': 1, 'kernel': 'linear'}

In [22]:
clf.cv_results_

{'mean_fit_time': array([0.0012351 , 0.00144749, 0.00099983, 0.00080056, 0.00103703,
        0.00059934]),
 'std_fit_time': array([0.00038391, 0.00055306, 0.00063203, 0.00040028, 0.00031307,
        0.00020042]),
 'mean_score_time': array([0.00102897, 0.00020113, 0.00060029, 0.00059986, 0.00056396,
        0.00030017]),
 'std_score_time': array([0.00015917, 0.00040226, 0.00049014, 0.00048978, 0.00024168,
        0.00024509]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['linear', 'rbf', 'linear', 'rbf', 'linear', 'rbf'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'linear'},
  {'C': 1, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'}],


In [24]:
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001235,0.000384,0.001029,0.000159,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.001447,0.000553,0.000201,0.000402,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,0.966667,0.966667,0.933333,1.0,0.966667,0.021082,5
2,0.001000,0.000632,0.000600,0.000490,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.966667,1.0,0.973333,0.038873,4
3,0.000801,0.000400,0.000600,0.000490,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
4,0.001037,0.000313,0.000564,0.000242,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.933333,1.0,0.966667,0.042164,5
5,0.000599,0.000200,0.000300,0.000245,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1


In [25]:
df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,linear,0.980000
1,1,rbf,0.966667
2,10,linear,0.973333
3,10,rbf,0.980000
4,20,linear,0.966667
5,20,rbf,0.980000


In [29]:
from sklearn.model_selection import RandomizedSearchCV
rclf = RandomizedSearchCV(svm.SVC(),parameters,n_iter=3)
rclf.fit(iris.data, iris.target)
rdf = pd.DataFrame(rclf.cv_results_)
rdf[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,20,linear,0.966667
1,10,rbf,0.980000
2,20,rbf,0.980000
